In [2]:
from Clasificador import Clasificador 
import numpy as np
import math 
from datos import Datos
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score 
from sklearn.model_selection import KFold
import EstrategiaParticionado
from EstrategiaParticionado import ValidacionCruzada



In [3]:
def normalizar_atributo(datos_atributo,esDiscreto,mean,std):
    if (esDiscreto):
        return datos_atributo
    else:
        return list(map(lambda x: (x - mean) / std, datos_atributo))
    
def comparativa_con_sklearn(datos):
    estrategia = EstrategiaParticionado.ValidacionCruzada(10)
    sklearn_cv = KFold(n_splits=10,shuffle=True)
    for k in [1,3,5,11,21,51]:
            knn_classifier = KNeighborsClassifier(n_neighbors=k,algorithm='ball_tree')
            clasificador = ClasificadorKNN(k)
            scores = cross_val_score(knn_classifier, datos.datos[:,:-1],datos.datos[:,-1], cv = sklearn_cv)
            errores = clasificador.validacion(estrategia,datos,clasificador)
            print ("Tasa de acierto k = ",k," :",1 - np.mean(errores))
            print ("Tasa de acierto k = ",k," version sklearn :", np.mean(scores))
            
class ClasificadorKNN (Clasificador):
    datos = np.array([])
    means = []
    stdevs = []
    k = 0
    def __init__(self,k):
        super(ClasificadorKNN,self).__init__()
        self.k = k
            
    def entrenamiento(self, datosTrain, atributosDiscretos, diccionario):
        datos_traspuestos = datosTrain.transpose()
        self.means = list(map(np.mean,datos_traspuestos))
        self.stdevs = list(map(np.std,datos_traspuestos))
        datos_normalizados = map(normalizar_atributo,
                                 datos_traspuestos,
                                 atributosDiscretos,
                                 self.means,
                                 self.stdevs)
        self.datos = np.array(list(datos_normalizados)).transpose()
  
    def __clasifica_dato(self,dato, atributosDiscretos, diccionario):
        def __dist_euclides(dato_entrenamiento):
            def dist_eje (x,y,esDiscreto):
                return int (x != y) if esDiscreto else pow((x - y),2)
            return math.sqrt(sum(map(dist_eje,
                                     dato_entrenamiento,
                                     dato,
                                     atributosDiscretos)))         
        distancias = map(__dist_euclides,self.datos[:,:-1])
        distancias_min = sorted(zip(distancias,self.datos[:,-1]),key = lambda x: x[0])[:self.k]
        vecinos = list(map(lambda x : x[1], distancias_min))
        dominio_clase = list(diccionario[-1].values())
        frecuencias_clases = list(map(vecinos.count,dominio_clase))
        return dominio_clase[frecuencias_clases.index(max(frecuencias_clases))]
                    
    def clasifica (self, datosTest, atributosDiscretos, diccionario):
        datos_test_normalizados = np.array(list(map(normalizar_atributo,
                                                    datosTest.transpose(),
                                                    atributosDiscretos,
                                                    self.means,
                                                    self.stdevs))).transpose()
        return [ self.__clasifica_dato (dato,atributosDiscretos,diccionario)
                 for dato in datos_test_normalizados]

    


In [8]:
wine = Datos ("../ficheros/ConjuntosDatos/wine_proc.data", True)
digits = Datos("../ficheros/ConjuntosDatos/digits.data",True)
example3 = Datos("../ficheros/ConjuntosDatos/example3.data",True)
example4 = Datos("../ficheros/ConjuntosDatos/example4.data",True)
print(len(wine.datos))

            

178


In [10]:
comparativa_con_sklearn(wine)

Tasa de acierto k =  1  : 0.375529411765
Tasa de acierto k =  1  version sklearn : 0.337581699346
Tasa de acierto k =  3  : 0.424235294118
Tasa de acierto k =  3  version sklearn : 0.370588235294
Tasa de acierto k =  5  : 0.373411764706
Tasa de acierto k =  5  version sklearn : 0.416993464052
Tasa de acierto k =  11  : 0.406823529412
Tasa de acierto k =  11  version sklearn : 0.402941176471
Tasa de acierto k =  21  : 0.416470588235
Tasa de acierto k =  21  version sklearn : 0.46568627451
Tasa de acierto k =  51  : 0.367529411765
Tasa de acierto k =  51  version sklearn : 0.364705882353


In [5]:
comparativa_con_sklearn(digits)

Tasa de acierto k =  1  : 0.2
Tasa de acierto k =  1  version sklearn : 0.198936170213
Tasa de acierto k =  3  : 0.160638297872
Tasa de acierto k =  3  version sklearn : 0.172340425532
Tasa de acierto k =  5  : 0.184042553191
Tasa de acierto k =  5  version sklearn : 0.187234042553
Tasa de acierto k =  11  : 0.191489361702
Tasa de acierto k =  11  version sklearn : 0.207446808511
Tasa de acierto k =  21  : 0.181914893617
Tasa de acierto k =  21  version sklearn : 0.210638297872
Tasa de acierto k =  51  : 0.172340425532
Tasa de acierto k =  51  version sklearn : 0.257446808511


In [7]:
comparativa_con_sklearn(example3)

Tasa de acierto k =  1  : 0.5025
Tasa de acierto k =  1  version sklearn : 0.5175
Tasa de acierto k =  3  : 0.5425
Tasa de acierto k =  3  version sklearn : 0.54
Tasa de acierto k =  5  : 0.485
Tasa de acierto k =  5  version sklearn : 0.5
Tasa de acierto k =  11  : 0.4625
Tasa de acierto k =  11  version sklearn : 0.435
Tasa de acierto k =  21  : 0.4075
Tasa de acierto k =  21  version sklearn : 0.4025
Tasa de acierto k =  51  : 0.44
Tasa de acierto k =  51  version sklearn : 0.43


In [9]:
comparativa_con_sklearn(example4)

Tasa de acierto k =  1  : 0.52
Tasa de acierto k =  1  version sklearn : 0.61
Tasa de acierto k =  3  : 0.55
Tasa de acierto k =  3  version sklearn : 0.57
Tasa de acierto k =  5  : 0.57
Tasa de acierto k =  5  version sklearn : 0.59
Tasa de acierto k =  11  : 0.53
Tasa de acierto k =  11  version sklearn : 0.48
Tasa de acierto k =  21  : 0.53
Tasa de acierto k =  21  version sklearn : 0.5
Tasa de acierto k =  51  : 0.47
Tasa de acierto k =  51  version sklearn : 0.47
